In [230]:
import requests
from bs4 import BeautifulSoup

import numpy as np

import pandas as pd

import re

import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


def extract_songs(artist):
    artist_url = 'https://www.lyrics.com/artist/' + artist
    artist_html = requests.get(artist_url).text

    soup = BeautifulSoup(artist_html)
    songs = dict()
    square_bracket_pattern = ' [\[].*[\]]'
    link_constant = 'https://www.lyrics.com/'
    for song in soup.find_all('strong'):
        a = song.find('a')
        if a and not (re.findall(square_bracket_pattern, a.text)) :
            songs[a.text.lower()] = link_constant + a.get('href')
    songs_df = pd.DataFrame(columns=["Title", "Link"])
    songs_df['Title'] = songs.keys()
    songs_df['Link'] = songs.values()
    return songs_df


# imagine_dragons_url = 'https://www.lyrics.com/artist/Imagine-Dragons'
# imagine_dragons_html = requests.get(imagine_dragons_url).text

# imagine_dragons_soup = BeautifulSoup(imagine_dragons_html)

# imagine_dragons_df = extract_songs('Imagine-Dragons')
# imagine_dragons_df.to_csv("../data/imagine_dragons_songs.csv")

In [231]:
# linkin_park_df = extract_songs('Linkin-Park')
# linkin_park_df.to_csv("../data/linkin_park_songs.csv")

In [232]:
import tqdm

def extract_lyrics_from_url(url):

    # time.sleep(10)
    print("Extrating from ", url)
    soup = BeautifulSoup(requests.get(url).text)
    lyrics = ""
    lyrics_tag = soup.find('pre', attrs={'id' : 'lyric-body-text'})
    if lyrics_tag :
        for child in lyrics_tag.children:
            lyrics += child.text
    return lyrics

# extract_lyrics_from_url('https://www.lyrics.com/lyric/36854002/Linkin+Park/In+the+End')

# imagine_dragons_df["Lyrics"] = imagine_dragons_df['Link'].apply(extract_lyrics_from_url)


# imagine_dragons_df.to_csv("../data/imagine_dragons_lyrics_df.csv")


# linkin_park_df["Lyrics"] = linkin_park_df['Link'].apply(extract_lyrics_from_url)


# linkin_park_df.to_csv("../data/linkin_park_df_lyrics_df.csv")



imagine_dragons_df = pd.read_csv("../data/imagine_dragons_lyrics_df.csv")


linkin_park_df = pd.read_csv("../data/linkin_park_df_lyrics_df.csv")


df = pd.concat([imagine_dragons_df,linkin_park_df])


X = df[["Lyrics"]]
X = X.assign(Lyrics = X["Lyrics"].astype(str))
X['Lyrics'] = X['Lyrics'].apply(lambda x : x.replace("\r", ""))
X



y_true = pd.Series([1] * imagine_dragons_df.shape[0] + [0] * linkin_park_df.shape[0])

X_train, X_test, y_train, y_test = train_test_split(X, y_true, random_state=42)


vectorizer = CountVectorizer(lowercase=True, stop_words='english', token_pattern='[A-Za-z]+', ngram_range=(1,1))
# # vectorizer = CountVectorizer(lowercase=True, stop_words='english', analyzer='word', ngram_range=(1,1))

X_train_cv = vectorizer.fit_transform(X_train["Lyrics"])
# df_bow_sklearn = pd.DataFrame(X_train_cv.toarray(), columns=vectorizer.get_feature_names_out())
# df_bow_sklearn.head()


logreg_cv = LogisticRegression(max_iter=6000)

logreg_cv.fit(X_train_cv, y_train)

X_test_cv = vectorizer.transform(X_test["Lyrics"])

logreg_cv.score(X_test_cv, y_test)



0.7746478873239436